In [ ]:
import numpy as np
import re

class Foo:
    def __init__(self):
        self.names = []
        self.equations = []
        self.data = []
    
    def open_file(self, path_file):
            self.data = []
            with open(path_file, "r") as f:
                for line in f:
                    self.data.append(line.strip())
                    
    def parse_lines(self):
        for line in self.data:
            left, right = line.split("=>")
            
            equation = {}
                        
            for elem in left.strip().split(", "):
                amount, name = elem.strip().split(" ")
                if name not in self.names:
                    self.names.append(name)
                equation[name] = amount
                
            for elem in right.strip().split(", "):
                amount, name = elem.strip().split(" ")
                if name not in self.names:
                    self.names.append(name)
                equation[name] = -int(amount)
            
            self.equations.append(equation)
        
    def solve_np(self, name_para, quantity):
        i_name = self.names.index(name_para)
        nb_param = len(self.names)
        self.equations.append({name_para: 1})
        matrix = np.zeros((nb_param, nb_param))
        for i, equation in enumerate(self.equations):
            for name, amount in equation.items():
                matrix[i, self.names.index(name)] = amount
        
        values = np.zeros((nb_param))
        values[self.names.index(name_para)] = quantity # for the recipe Fuell = 1
        print(matrix)
        print(values)
        solutions = np.linalg.solve(matrix, values)
        
        return solutions[i_name]
    
            
                

In [ ]:
f = Foo()
f.open_file("/home/bchatillon/Documents/Advent-of-Code/2019/14_data_test1.txt")
f.parse_lines()


In [ ]:
f.solve_np("ORE", 1/1e12)

In [ ]:
import math
from collections import defaultdict

class FuelProducer:
    def __init__(self):
        self.equations = {}
        self.data = []
        self.dependences = {}
    
    def open_file(self, path_file):
            self.data = []
            with open(path_file, "r") as f:
                for line in f:
                    self.data.append(line.strip())
                    
    def parse_lines(self):
        for line in self.data:
            left, right = line.split("=>")
            
            composents = {}
                        
            for elem in left.strip().split(", "):
                amount, name = elem.strip().split(" ")
                composents[name] = int(amount)
                
            produce_amount, produce_name = right.strip().split(" ")
            
            self.equations[produce_name] = {"amount":int(produce_amount), "composents":composents}
            
            
    def get_ore_greedy(self):
        inventory = [("FUEL", 1)] # elem --> amount needed
        excess = defaultdict(int)
        ore_count = 1e12
        nb_fuel = 0
        
        while(True):
            elem, amount = inventory.pop(0)
            
            if excess[elem]:
                new_amount = max(amount-excess[elem], 0)
                excess[elem] -= (amount - new_amount)
                amount = new_amount
            if elem == "ORE":
                if amount > ore_count:
                    return nb_fuel
                ore_count -= amount
            elif amount != 0:
                recipe_elem = self.equations[elem]
                mult = math.ceil(amount/recipe_elem["amount"])
                excess[elem] += (mult*recipe_elem["amount"]-amount)

                for compo_name, compo_amount in recipe_elem["composents"].items():
                    amount_needed = mult * compo_amount
                    inventory.append((compo_name, amount_needed))
            
            if len(inventory) == 0:
                nb_fuel+=1
                inventory.append(("FUEL", 1))
                    
    def get_ore(self, elem, number, debug=False):
        inventory = [(elem, number)] # elem --> amount needed
        excess = defaultdict(int)
        usage = defaultdict(int)
        ore_count = 0
        
        while(True):
            if debug:
                print(inventory)
                print(excess)
                print()
            elem, amount = inventory.pop(0)
            
            if excess[elem]:
                new_amount = max(amount-excess[elem], 0)
                excess[elem] -= (amount - new_amount)
                amount = new_amount
            if elem == "ORE":
                ore_count += amount
            elif amount != 0:
                recipe_elem = self.equations[elem]
                mult = math.ceil(amount/recipe_elem["amount"])
                excess[elem] += (mult*recipe_elem["amount"]-amount)

                for compo_name, compo_amount in recipe_elem["composents"].items():
                    amount_needed = mult * compo_amount
                    inventory.append((compo_name, amount_needed))
            
            if len(inventory) == 0:
                break
        return ore_count, excess
    
    def part_1(self, debug=False):
        return self.get_ore("FUEL", 1, debug)[0]
            
    def part_2(self, debug=False):
        return self.get_ore_greedy()
    
    def part_2_bis(self):
        self.part_1
        

In [ ]:
fp = FuelProducer()
fp.open_file("/home/bchatillon/Documents/Advent-of-Code/2019/14_data.txt")
fp.parse_lines()
fp.equations


In [ ]:
fp.part_1(debug = False)

In [ ]:
fp.part_2()

In [ ]:
a = collections.Counter()